In [1]:
# API KEY를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API KEY 정보로드
load_dotenv()

True

설치된 openai 의 버전을 체크합니다.


In [2]:
import openai

openai.__version__

'1.6.0'

## ChatOpenAI

OpenAI 사의 채팅 전용 Large Language Model(llm) 입니다.

객체를 생성할 때 다음을 옵션 값을 지정할 수 있습니다. 옵션에 대한 상세 설명은 다음과 같습니다.

`temperature`

- 사용할 샘플링 온도는 0과 2 사이에서 선택합니다. 0.8과 같은 높은 값은 출력을 더 무작위하게 만들고, 0.2와 같은 낮은 값은 출력을 더 집중되고 결정론적으로 만듭니다.

`max_tokens`

- 채팅 완성에서 생성할 토큰의 최대 개수입니다.

`model_name`: 적용 가능한 모델 리스트

- 링크: https://platform.openai.com/docs/models


In [10]:
from langchain_openai import ChatOpenAI

# 객체 생성
llm = ChatOpenAI(
    temperature=0.1,  # 창의성 (0.0 ~ 2.0)
    max_tokens=2048,  # 최대 토큰수
    model_name="gpt-4o",  # 모델명
)

# 질의내용
question = """질문에 대하여 약 100자 내외로 답변해 주세요.
답변에는 반드시 이모지를 넣어주세요.
질문 : 대한민국의 수도는 어디인가요?
"""

# 질의
# print(f"[답변]: {llm.invoke(question)}")

In [11]:
answer = llm.invoke(question)

In [12]:
print(answer.content)

대한민국의 수도는 서울입니다. 🏙️ 서울은 정치, 경제, 문화의 중심지로 다양한 명소가 많습니다. 🌟


## 프롬프트 템플릿의 활용


`PromptTemplate`

- 사용자의 입력 변수를 사용하여 완전한 프롬프트 문자열을 만드는 데 사용되는 템플릿입니다
- 사용법
  - `template`: 템플릿 문자열입니다. 이 문자열 내에서 중괄호 `{}`는 변수를 나타냅니다.
  - `input_variables`: 중괄호 안에 들어갈 변수의 이름을 리스트로 정의합니다.

`input_variables`

- input_variables는 PromptTemplate에서 사용되는 변수의 이름을 정의하는 리스트입니다.
- 사용법: 리스트 형식으로 변수 이름을 정의합니다.


In [13]:
from langchain.prompts import PromptTemplate

# 질문 템플릿 형식 정의
template = "{country}의 수도는 뭐야?"

# 템플릿 완성
prompt = PromptTemplate.from_template(template=template)
prompt

PromptTemplate(input_variables=['country'], template='{country}의 수도는 뭐야?')

### LLMChain 객체


`LLMChain`

- LLMChain은 특정 PromptTemplate와 연결된 체인 객체를 생성합니다
- 사용법
  - `prompt`: 앞서 정의한 PromptTemplate 객체를 사용합니다.
  - `llm`: 언어 모델을 나타내며, 이 예시에서는 이미 어딘가에서 정의된 것으로 보입니다.


In [23]:
from langchain.chains import LLMChain

# 연결된 체인(Chain)객체 생성
llm_chain = LLMChain(prompt=prompt, llm=llm)

# 프롬프트, 모델, 출력 파서를 연결하여 처리 체인을 구성합니다.
chain = prompt | model | output_parser

NameError: name 'model' is not defined

In [21]:
llm_chain.invoke({"country": "대한민국"})

대한민국의 수도는 서울이야.

{'country': '대한민국', 'text': '대한민국의 수도는 서울이야.'}

In [16]:
llm_chain.invoke({"country": "캐나다"})

{'country': '캐나다',
 'text': '캐나다의 수도는 오타와입니다. 오타와는 온타리오주에 위치해 있으며, 캐나다의 정치적 중심지로서 중요한 역할을 하고 있습니다.'}

### apply()

`apply()` 함수로 여러개의 입력에 대한 처리를 한 번에 수행할 수 있습니다.


In [8]:
input_list = [{"country": "호주"}, {"country": "중국"}, {"country": "네덜란드"}]

response = llm_chain.apply(input_list)

In [9]:
response[0]["text"]

'호주의 수도는 캔버라입니다.'

`text` 키 값으로 결과 뭉치가 반환되었음을 확인할 수 있습니다.

이를 반복문으로 출력한다면 다음과 같습니다.


In [10]:
# input_list 에 대한 결과 반환
result = llm_chain.apply(input_list)

# 반복문으로 결과 출력
for res in result:
    print(res["text"].strip())

호주의 수도는 캔버라(Canberra)입니다.
중국의 수도는 베이징(北京)입니다.
네덜란드의 수도는 암스테르담입니다.


### `generate()`


`generate()` 는 문자열 대신에 LLMResult를 반환하는 점을 제외하고는 apply와 유사합니다.

LLMResult는 토큰 사용량과 종료 이유와 같은 유용한 생성 정보를 자주 포함하고 있습니다.


In [11]:
# 입력값
input_list = [{"country": "호주"}, {"country": "중국"}, {"country": "네덜란드"}]

# input_list 에 대한 결과 반환
generated_result = llm_chain.generate(input_list)
print(generated_result)

generations=[[ChatGeneration(text='호주의 수도는 캔버라입니다.', generation_info={'finish_reason': 'stop', 'logprobs': None}, message=AIMessage(content='호주의 수도는 캔버라입니다.'))], [ChatGeneration(text='중국의 수도는 베이징(北京)입니다.', generation_info={'finish_reason': 'stop', 'logprobs': None}, message=AIMessage(content='중국의 수도는 베이징(北京)입니다.'))], [ChatGeneration(text='네덜란드의 수도는 암스테르담입니다.', generation_info={'finish_reason': 'stop', 'logprobs': None}, message=AIMessage(content='네덜란드의 수도는 암스테르담입니다.'))]] llm_output={'token_usage': {'completion_tokens': 53, 'prompt_tokens': 58, 'total_tokens': 111}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_4f0b692a78'} run=[RunInfo(run_id=UUID('804b0b7e-16c3-4dd1-b1c4-c9574cb5f0f1')), RunInfo(run_id=UUID('3e1ba16a-65c5-46b9-8b3e-44ceb10e7cb6')), RunInfo(run_id=UUID('34fc2b22-227d-41f3-a8de-3e6a3ffd5eca'))]


In [12]:
# 토큰 사용량 출력
generated_result.llm_output

{'token_usage': {'completion_tokens': 53,
  'prompt_tokens': 58,
  'total_tokens': 111},
 'model_name': 'gpt-3.5-turbo',
 'system_fingerprint': 'fp_4f0b692a78'}

### 2개 이상의 변수를 템플릿 안에 정의


2개 이상의 변수를 적용하여 템플릿을 생성할 수 있습니다.

이번에는 2개 이상의 변수(`input_variables`) 를 활용하여 템플릿 구성을 해보겠습니다.


In [13]:
# 질문 템플릿 형식 정의
template = "{area1} 와 {area2} 의 시차는 몇시간이야?"

# 템플릿 완성
prompt = PromptTemplate.from_template(template)
prompt

PromptTemplate(input_variables=['area1', 'area2'], template='{area1} 와 {area2} 의 시차는 몇시간이야?')

In [14]:
# 연결된 체인(Chain)객체 생성
llm_chain = LLMChain(prompt=prompt, llm=llm)

In [19]:
# 체인 실행: run()
print(llm_chain.invoke({"area1": "서울", "area2": "파리"}))

ValueError: Missing some input keys: {'country'}

In [16]:
input_list = [
    {"area1": "파리", "area2": "뉴욕"},
    {"area1": "서울", "area2": "하와이"},
    {"area1": "켄버라", "area2": "베이징"},
]

# 반복문으로 결과 출력
result = llm_chain.apply(input_list)
for res in result:
    print(res["text"].strip())

파리와 뉴욕의 시차는 6시간입니다. 파리는 그리니치 평균시(GMT+1)를 따르고, 뉴욕은 동부 표준시(EST)를 따르기 때문에 시차가 발생합니다.
서울과 하와이의 시차는 서울이 하와이보다 19시간 빠릅니다. 서울은 GMT+9 시간대에 있고, 하와이는 GMT-10 시간대에 있기 때문입니다.
켄버라와 베이징의 시차는 2시간입니다. 켄버라는 UTC+10 시간대에 속하고, 베이징은 UTC+8 시간대에 속하기 때문입니다.


## stream: 실시간 출력

스트리밍 옵션은 질의에 대한 답변을 실시간으로 받을 때 유용합니다.

다음과 같이 `streaming=True` 로 설정하고 스트리밍으로 답변을 받기 위한 `StreamingStdOutCallbackHandler()` 을 콜백으로 지정합니다.


In [17]:
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

# 객체 생성
llm = ChatOpenAI(
    temperature=0,  # 창의성 (0.0 ~ 2.0)
    max_tokens=2048,  # 최대 토큰수
    model_name="gpt-3.5-turbo",  # 모델명
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()],
)

In [18]:
# 질의내용
question = "대한민국에 대해서 300자 내외로 최대한 상세히 알려줘"

# 스트리밍으로 답변 출력
response = llm.invoke(question)

대한민국은 동아시아에 위치한 나라로, 수도는 서울이며 인구는 약 5천만 명이 넘는 대국이다. 대한민국은 민주공화국으로 정부는 대통령 중심의 체제를 가지고 있으며, 국회는 한국 국회로 불린다. 대한민국은 한반도의 남쪽에 위치하고 있으며 북쪽으로는 북한과 접하고 있다.

대한민국은 세계에서 11번째로 경제 규모가 큰 나라로, 선진국으로 분류되는 나라 중 하나이다. 주요 산업은 반도체, 자동차, 조선, 철강, 전자제품 등이 있으며, 세계적으로 유명한 기업들이 많이 발달해 있다.

문화적으로는 한류가 세계적으로 유명하며, 한국의 음식, 의상, 음악, 영화 등이 전 세계에서 사랑받고 있다. 또한 한국의 전통문화인 한복, 한국 무용, 사물놀이, 판소리 등도 많은 사람들에게 인기가 있다.

대한민국은 교육 수준이 높고 고등 기술력이 발달한 나라로, 세계적으로 인정받는 대학들이 많이 있다. 또한 대한민국은 안전하고 깨끗한 환경을 유지하고 있어 여행객들에게도 매력적인 나라로 손꼽힌다.